In [ ]:
import openai
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your_api_key"
openai.api_key = os.getenv("OPENAI_API_KEY")

def convert_to_image_prompt(summary: str) -> str:
    """
    Converts a text summary into an image prompt using OpenAI's API.
    
    Args:
    - summary (str): The text summary of the welfare scheme section.
    
    Returns:
    - str: The generated image prompt.
    """
    prompt = (
        f"Convert the following text summary into a detailed, descriptive image prompt for "
        f"a text-to-image generative model. This generative model will be used to create a poster for the welfare scheme mentioned especially for people who don't speak english. Do not add text to the poster. Make sure to include vivid visual elements, "
        f"colors, expressions, and any relevant scene settings:\n\n"
        f"Summary: {summary}\n\nImage prompt:"
    )

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant who has to convert a text summary into an image prompt for a text-to-image generative model. This generative model will be used to create a poster for the welfare scheme mentioned. Do not add text to the poster, this would make the poster more accessible to people who cannot read. ENSURE THAT THE PROMPT IS COMPLIANT WITH OPENAI'S POLICIES AND DOES NOT CONTAIN OFFENSIVE INFORMATION."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.7
    )

    image_prompt = response.choices[0].message.content
    return image_prompt


In [2]:
import os

summaries_directory = "scheme_writeups_gpt"
image_prompts_directory = "image_prompts"

os.makedirs(image_prompts_directory, exist_ok=True)

def generate_image_prompts_from_summaries(summaries_dir, prompts_dir):
    scheme_prompts = {}

    for filename in os.listdir(summaries_dir):
        if filename.endswith(".txt"):
            
            scheme_name, section_name = filename.split('_', 1)
            section_name = section_name.replace('.txt', '').replace('_', ' ')
            
            file_path = os.path.join(summaries_dir, filename)
            
            with open(file_path, "r") as summary_file:
                summary_content = summary_file.read()
                
            if scheme_name not in scheme_prompts:
                scheme_prompts[scheme_name] = []
            scheme_prompts[scheme_name].append((section_name, summary_content))
    
    for scheme_name, sections in scheme_prompts.items():
        full_summary = "\n".join([f"{section}: {content}" for section, content in sections])
        
        image_prompt = convert_to_image_prompt(full_summary)        
        prompt_filename = f"{scheme_name}.txt"
        prompt_file_path = os.path.join(prompts_dir, prompt_filename)
        
        with open(prompt_file_path, "w") as prompt_file:
            prompt_file.write(image_prompt)
            
        print(f"Image prompt saved for scheme: {scheme_name} as {prompt_filename}")

generate_image_prompts_from_summaries(summaries_directory, image_prompts_directory)

Image prompt saved for scheme: MGNREGA as MGNREGA.txt
Image prompt saved for scheme: ABPMJAY as ABPMJAY.txt
Image prompt saved for scheme: NRLM as NRLM.txt
Image prompt saved for scheme: AAY as AAY.txt
Image prompt saved for scheme: DDUGKY as DDUGKY.txt
